In [ ]:
import scipy.linalg

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%matplotlib widget
# aka import ipympl

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from mpl_toolkits.mplot3d import axes3d, Axes3D

In [ ]:
points_3d = pd.read_csv("WS_measures/df_B3_Etat_Initial.csv")

In [ ]:
groupby_name = points_3d.groupby(['name'])

In [ ]:
#fig = plt.figure(figsize=(6,6))
fig = plt.figure()

ax = fig.gca(projection='3d')
#ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
for name,group in groupby_name:
    #group.plot(x='saleDate', y='MeanToDate', title=title)
    ax.scatter(group.x, group.y, group.z, c=group.color, s=50, label=group.name.unique())


ax.legend(loc='best')

In [ ]:
def fit3dPlane(df, coords=["x","y","z"], order=1, x_bound=None, y_bound=None, \
               doPlot=False, plot_path=None, plot_title=None, plot_name=None, savePlot=False):
#def fit3dPlane(df, coords=["x","y","z"], order=1, doPlot=False, plot_path=None, exp=None):
    
    import scipy.linalg
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    
    x = df[coords[0]]
    y = df[coords[1]]
    z = df[coords[2]]
    x_bound = [x.min(),x.max()] if x_bound is None else x_bound
    y_bound = [y.min(),y.max()] if y_bound is None else y_bound

    # regular grid covering the domain of the data
    X,Y = np.meshgrid(np.arange(x_bound[0], x_bound[1], 100), np.arange(y_bound[0], y_bound[1], 100))
    XX = X.flatten()
    YY = Y.flatten()

    # 1: linear, 2: quadratic
    if order == 1:
        # best-fit linear plane
        A = np.c_[x, y, np.ones(x.shape[0])]
        C,_,_,_ = scipy.linalg.lstsq(A, z)    # coefficients

        # evaluate it on grid
        Z = C[0]*X + C[1]*Y + C[2]

        # or expressed using matrix/vector product
        #Z = np.dot(np.c_[XX, YY, np.ones(XX.shape)], C).reshape(X.shape)

    elif order == 2:
        # best-fit quadratic curve
        A = np.c_[np.ones(x.shape[0]), df[[coords[0],coords[1]]], \
                  np.prod(df[[coords[0],coords[1]]].values, axis=1),df[[coords[0],coords[1]]].values**2]
        C,_,_,_ = scipy.linalg.lstsq(A, z)
        # evaluate it on a grid
        Z = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX*YY, XX**2, YY**2], C).reshape(X.shape)

    if doPlot:
    # plot points and fitted surface
        fig = plt.figure()
        ax = fig.gca(projection='3d')
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
        ax.scatter(x, y, z, c='r', s=50)
        plt.xlabel('X')
        plt.ylabel('Y')
        ax.set_zlabel('Z')
#        ax.axis('equal')
#        ax.axis('tight')

        if plot_title is not None:
            plt.suptitle(plot_title)

        if savePlot :
            if plot_path is None or plot_title is None:
                  raise Exception("you must specify a plot path and a title")
            plt.savefig(plot_path+plot_title)
        plt.show()
#        plt.cla()
    
    return C


In [ ]:
def getBestPlane(data, order=1, doPlot=False, plot_path=None, exp=None):
    
    coords = ["px", "py", "relPos"]
    x_bound = [0,4200]
    y_bound = [0,4200]
    
    if exp is not None:
        plot_title = f"Focus_plane_Exp{exp}.png"
        plot_name = f"Exp{exp}"
    else :
        plot_title = None
        plot_name = None

    return fit3dPlane(df, coords=coords, order=1, x_bound=x_bound, y_bound=y_bound, \
                      doPlot=doPlot, plot_path=plot_path, plot_title=plot_title, plot_name=plot_name)


In [ ]:
group =  groupby_name.get_group("B3")


fit3dPlane(group, doPlot=True, order=1)

In [ ]:
from scipy.linalg import norm

In [ ]:
np.cross(z)

In [ ]:
group_liste = ["B1", "B2", "B3"]
groups = pd.concat( [ groupby_name.get_group(gr) for gr in group_liste] )

fit3dPlane(groups, doPlot=True)

In [ ]:
group_liste = ["Plan DetBox"]

groups = pd.concat( [ groupby_name.get_group(gr) for gr in group_liste] ).groupby('name')    
fig = plt.figure()
ax = fig.gca(projection='3d')
#ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
for name,group in groups:
    #group.plot(x='saleDate', y='MeanToDate', title=title)
    ax.scatter(group.x, group.y, group.z, c=group.color, s=50, label=group.name.unique())
#Axes3D.plot3D(0,0,0)
ax.legend()
fit3dPlane(group, doPlot=False)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)